In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
tfb = tfp.bijectors

In [3]:
class MySigmoid(tfb.Bijector):
    
    def __init__(self, validate_args=False, name='sigmoid'):
        super(MySigmoid, self).__init__(
            validate_args=validate_args, forward_min_event_ndims=0, name=name)

We are passing the argument `forward_min_event_ndims` to the class initializer. When you are creating a bijector you need to set the minimum number of event dimensions that the bijector needs to act upon. In this case, the sigmoid is a function that can operate on scalars, so the minimum number of dimensions is set to zero.

For instance, in the softmax centered bijector the inputs need to have at least rank 1. 

You need to supply the `forward_min_event_ndims` argument of the `inverse_min_event_ndims` argument. In most cases you just need to supply one of them to the base class constructor. However, some bijectors might change the shape of the input, so it is possible that the minimum number of event dimensions is different in the forward and in the inverse transformations.

In [4]:
class MySigmoid(tfb.Bijector):
    
    def __init__(self, validate_args=False, name='sigmoid'):
        super(MySigmoid, self).__init__(
            validate_args=validate_args, forward_min_event_ndims=0, name=name)
        
    def _forward(self, x):
        return tf.math.sigmoid(x)
    
    def _inverse(self, y):
        return tf.math.log(y) - tf.math.log(1-y)

Next, we need to define the forward and inverse tranformations of the bijector. These should be implemented as the `_forward` and `_inverse` methods of the class. In this example, we are using the built in sigmoid function. The inverse of the sigmoid is the logit function.

In [5]:
class MySigmoid(tfb.Bijector):
    
    def __init__(self, validate_args=False, name='sigmoid'):
        super(MySigmoid, self).__init__(
            validate_args=validate_args, forward_min_event_ndims=0, name=name)
        
    def _forward(self, x):
        return tf.math.sigmoid(x)
    
    def _inverse(self, y):
        return tf.math.log(y) - tf.math.log(1-y)
    
    def _inverse_log_det_jacobian(self, y):
        return -tf.math.log(y) - tf.math.log(1-y)
    
    def _forward_log_det_jacobian(self, y):
        return -tf.math.softplus(-x) - tf.math.log(x)
        #return -self._inverse_log_det_jacobian(self._forward(x))

The log jacobian computation should return a scalar for each event.

Besides we know that the inverse of the log jacobian determinant is just the inverse of the forward log jacobian, so we could, for instance, implement the forward log det jacobian using this fact. So, in fact, we only need to supply one of the log det jacobian methods and the inverse will be computed automatically in the form written in a comment above.

In [6]:
class MyShift(tfb.Bijector):
    
    def __init__(self, shift, validate_args=False, name='shift'):
        self.shift = shift
        super(MyShift, self).__init__(
        validate_args=validate_args, forward_min_event_ndims=0, name=name,
        is_constant_jacobian=True)
        
    def _forward(self, x):
        return x + self.shift
    
    def _inverse(self, y):
        return y - self, shift
    
    def _forward_log_det_jacobian(self, x):
        return tf.constant(0, x.dtype)

The keyword argument `is_constant_jacobian` can be set to true when the log jacobian determinant of the bijector is independent of the input. An example is showned above for a simple shift bijector. In this case the jacobian is a constant zero.